In [ ]:
%cd ..

In [ ]:
from copy import deepcopy
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import warnings

import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
from carla.data.api import data
import numpy as np
import torch
torch.cuda.is_available = lambda : False
import yaml
from seed_env import seed_my_session
from typing import Dict, List
from cent.data_specific import DataModels
from carla import Benchmark
import pandas as pd
from carla.recourse_methods import (
    CCHVAE,
    CEM,
    CRUD,
    FOCUS,
    CausalRecourse,
    Clue,
    Dice,
    Face,
    FeatureTweak,
    GrowingSpheres,
    Revise,
    Wachter,
)
from carla.recourse_methods.catalog.causal_recourse import constraints, samplers
import carla.evaluation.catalog as evaluation_catalog
from cent.method import CEnt
from vae_benchmark import VAEBenchmark
from tensorflow import Graph, Session
from carla.models.catalog import MLModelCatalog

In [ ]:
data_name = 'adult'
FACTUAL_NUMBER = 20
OUT_DIR_DATA = 'outputs/tmp/'
if not os.path.exists(OUT_DIR_DATA):
    os.makedirs(OUT_DIR_DATA)
data_models = DataModels(data_name = data_name,
                             factuals_length = FACTUAL_NUMBER,
                             out_dir = OUT_DIR_DATA)

In [5]:
layers = [25]
latent_dim = 7
mlmodel = data_models.models_zoo['ann']['tensorflow']

In [ ]:
min_entries_per_label = int(data_models.trainData.df.shape[0]*0.01)
MIN_ENTRIES_PER_LABEL_THRESH = 500
if min_entries_per_label<MIN_ENTRIES_PER_LABEL_THRESH:
    print('min_entries_per_label is too small {}, setting it to {} '.format(min_entries_per_label,MIN_ENTRIES_PER_LABEL_THRESH))
#TODO: @MM Return this to 1% of data
min_entries_per_label = MIN_ENTRIES_PER_LABEL_THRESH
hpr = {"data_name": "data_name","n_search_samples": 300,
        "p_norm": 1,"step": 0.1,"max_iter": 10,"clamp": True,
        "treeWarmUp": 5,
        "binary_cat_features": True,
        "myvae_params": {
            'input_dim': len(mlmodel.feature_input_order),
            'kld_weight': 0.00025,
            'layers': layers,
            'latent_dim': latent_dim,
            'hidden_activation': 'relu',
            'dropout': 0.2,
            'batch_norm': True,
            'batch_size': 32,
            'epochs': 15,
            'learning_rate': 0.001,
            'weight_decay': 0.000001,
            'cuda': False,
            'verbose': True,
            'train': True,
            'save_dir': './vae_model/',
        },
        "tree_params": {
            "min_entries_per_label": min_entries_per_label,
            "grid_search_jobs": -1,
            "min_weight_gini": 100,
            "max_search" : 50,
            "grid_search": {"cv": 1,"splitter": ["best"],"criterion": ["gini"],"max_depth": [3,4,5,6,7],
                            "min_samples_split": [1.0,2,3],"min_samples_leaf": [1,2,3],
                            "max_features": ['sqrt',1.0, 'log2',0.8],
                            }
        }
    }
cent_method =  CEnt(deepcopy(data_models.trainData), mlmodel, hpr, data_catalog= data_models.new_catalog_n)

In [7]:
from time import time
fact = data_models.factuals['ann'].sample(1)

In [8]:
start_timer = time()
contrast = cent_method.get_counterfactuals(fact)
print('Time taken to get contrast: {:.2f} seconds'.format(time()-start_timer))
contrast

Time taken to get contrast: 0.04 seconds


,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
32522,0.164384,0.076916,0.827853,0.0,0.304751,...,1.0,1.0,1.0,0.0,1.0


In [37]:
print("To flip the binary class from {} to {}, we moved our factual to a point satisfying the following conditions:".format(fact[mlmodel.data.target].values[0],fact[mlmodel.data.target].values[0]*-1 +1))
for rule in cent_method.found_node.conditions:
    if rule.feature in data_models.new_catalog_n['categorical']:
        to_bin_value = bool(fact['marital-status_Non-Married'].values[0]*-1+1)
        print('\t{} need to be flipped to {}'.format(rule.feature,to_bin_value))
    else:
        print('\t{} need to be {} than {}'.format(rule.feature,rule.threshold_sign, rule.threshold))

To flip the binary class from 0 to 1, we moved our factual to a point satisfying the following conditions:
	education-num need to be > than 0.8333333432674408
	marital-status_Non-Married need to be flipped to False
	capital-loss need to be > than 0.30486685037612915
